In [1]:
] activate ../

  Activating project at `/dfs6/pub/ikeshwan/code/HorizontalConvection`


In [2]:
] status

Status `/dfs6/pub/ikeshwan/code/HorizontalConvection/Project.toml`
⌃ [13f3f980] CairoMakie v0.10.10
  [7073ff75] IJulia v1.24.2
⌃ [b946abbf] NaNStatistics v0.6.31
⌃ [9e8cae18] Oceananigans v0.88.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [3]:
using Oceananigans
#using Printf

In [4]:
grid = RectilinearGrid(size=(128, 128), x=(-5, 5), z=(-5, 5),
                       topology=(Periodic, Flat, Bounded))

128×1×128 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CPU with 3×0×3 halo
├── Periodic x ∈ [-5.0, 5.0)      regularly spaced with Δx=0.078125
├── Flat y
└── Bounded  z ∈ [-5.0, 5.0]      regularly spaced with Δz=0.078125

In [17]:
shear_flow(x, y, z, t) = tanh(z)

stratification(x, y, z, t, p) = p.h * p.Ri * tanh(z / p.h)

U = BackgroundField(shear_flow)

B = BackgroundField(stratification, parameters=(Ri=0.1, h=1/4));

In [18]:
using CairoMakie

zF = znodes(grid, Face())
zC = znodes(grid, Center())

Ri, h = B.parameters

fig = Figure(size = (850, 450))

ax = Axis(fig[1, 1], xlabel = "U(z)", ylabel = "z", limits=((nothing, nothing), (-5, 5)))
lines!(ax, shear_flow.(0, zC, 0), zC; linewidth = 3)

ax = Axis(fig[1, 2], xlabel = "B(z)", limits=((nothing, nothing), (-5, 5)))
lines!(ax, [stratification(0, z, 0, (Ri=Ri, h=h)) for z in zC], zC; linewidth = 3, color = :red)

ax = Axis(fig[1, 3], xlabel = "Ri(z)", limits=((nothing, nothing), (-5, 5)))
lines!(ax, [Ri * sech(z / h)^2 / sech(z)^2 for z in zF], zF; linewidth = 3, color = :black) # Ri(z)= ∂_z B / (∂_z U)²; derivatives computed by hand

fig

LoadError: MethodError: no method matching shear_flow(::Int64, ::Float64, ::Int64)

[0mClosest candidates are:
[0m  shear_flow(::Any, ::Any, ::Any, [91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[17]:1[24m[39m


In [7]:
model = NonhydrostaticModel(timestepper = :RungeKutta3,
                              advection = UpwindBiasedFifthOrder(),
                                   grid = grid,
                               coriolis = nothing,
                      background_fields = (u=U, b=B),
                                closure = ScalarDiffusivity(ν=0.2e-4, κ=0.2e-4),
                               buoyancy = BuoyancyTracer(),
                                tracers = :b)

NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 128×1×128 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── tracers: b
├── closure: ScalarDiffusivity{ExplicitTimeDiscretization}(ν=2.0e-5, κ=(b=2.0e-5,))
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: Nothing

In [8]:
Δt = 0.1 
simulation = Simulation(model, Δt=Δt, stop_iteration=2000, verbose=false)

Simulation of NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 100 ms
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: Inf days
├── Stop iteration : 2000.0
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   └── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [9]:
using Random, Statistics

u, v, w = model.velocities
b = model.tracers.b
xb, yb, zb = nodes(b)
total_b = Field(b + model.background_fields.tracers.b)

mean_perturbation_kinetic_energy = Field(Average(1/2 * (u^2 + w^2)))

noise(x, y, z) = 1.e-3*randn()
set!(model, u=noise, w=noise, b=noise)

In [10]:
simulation.output_writers[:buoyancy] =
    JLD2OutputWriter(model, (b=b, B=total_b),
                     schedule = TimeInterval(1.0),
                     filename = "../output/stratified_shear_data/example.jld2",
                     overwrite_existing = true)

JLD2OutputWriter scheduled on TimeInterval(1 second):
├── filepath: ./../output/stratified_shear_data/example.jld2
├── 2 outputs: (b, B)
├── array type: Array{Float64}
├── including: [:grid, :coriolis, :buoyancy, :closure]
└── max filesize: Inf YiB

In [11]:
@info "*** Running a simulation of Kelvin-Helmholtz instability..."
run!(simulation)

[ Info: *** Running a simulation of Kelvin-Helmholtz instability...


In [12]:
@info "Making a neat movie of stratified shear flow..."

filepath = simulation.output_writers[:buoyancy].filepath

B_timeseries = FieldTimeSeries(filepath, "B")

times = B_timeseries.times
t_final = times[end]

n = Observable(1)

Bₙ = @lift interior(B_timeseries, :, 1, :, $n)

fig = Figure(size=(800, 600))

kwargs = (xlabel="x [m]", ylabel="z [m]", limits = ((xb[1], xb[end]), (-3, 3)), aspect=1,)

title = @lift @sprintf("temperature [°C] at t = %.2f", times[$n])

ax_B = Axis(fig[1, 1]; title = title, kwargs...)

B_lims = (-maximum(abs, interior(B_timeseries, :, 1, :, :)), maximum(abs, interior(B_timeseries, :, 1, :, :)))

hm_B = heatmap!(ax_B, xb, zb, Bₙ; colorrange = B_lims, colormap = :balance)
Colorbar(fig[1, 2], hm_B)

frames = 1:length(times)

record(fig, "../animations/shear_instability_example.mp4", frames, framerate=8) do i
    @info "Plotting frame $i of $(frames[end])..."
    n[] = i
end

[ Info: Making a neat movie of stratified shear flow...


LoadError: LoadError: UndefVarError: `@sprintf` not defined
in expression starting at In[12]:18

In [13]:
fig = Figure(size=(900, 600))
kwargs = (xlabel="x [m]", ylabel="z [m]", limits = ((xb[1], xb[end]), (zb[1], zb[end])), aspect=1,)
ax_B = Axis(fig[1, 1]; title = "temperature [°C] at t=0 minutes", kwargs...)
hm_B = heatmap!(ax_B, xb, zb, interior(B_timeseries, :, 1, :, 1); colorrange = B_lims, colormap = :balance)

ax_B = Axis(fig[1, 2]; title = "temperature [°C] at t=90 minutes", kwargs...)
hm_B = heatmap!(ax_B, xb, zb, interior(B_timeseries, :, 1, :, 90); colorrange = B_lims, colormap = :balance)
lines!(ax_B, [0, 0], [-5, 5], linestyle=:dash, color=:black)

ax_B = Axis(fig[1, 3]; title = "temperature [°C] at t=180 minutes", kwargs...)
hm_B = heatmap!(ax_B, xb, zb, interior(B_timeseries, :, 1, :, 180); colorrange = B_lims, colormap = :balance)

ax_B = Axis(fig[2, 2]; title = "temperature at t=90 minutes", xlabel="temperature [ºC]", ylabel="z [m]", limits=(0.75.*B_lims, (zb[1], zb[end])))
l_B = lines!(ax_B, interior(B_timeseries, 64, 1, :, 90), zb, label="x=0 m")

Colorbar(fig[1, 4], hm_B)
leg = axislegend(ax_B, position=:lt)
save("../figures/shear_instability_snapshots.png", fig)
#fig

l_B = lines!(ax_B, mean(interior(B_timeseries, :, 1, :, 90), dims=(1))[1,:], zb, label="zonal average")
delete!(leg)
axislegend(ax_B, position=:lt)
save("../figures/shear_instability_snapshots_mean.png", fig)
fig

LoadError: UndefVarError: `B_lims` not defined

In [14]:
using Statistics
Bₙ = interior(B_timeseries, :, 1, :, 186)
b_mean = mean(Bₙ, dims=(1))[1,:]

lines(b_mean, zb)

print(size(Bₙ))



(128, 128)

#### Now let's try to calculate $z_{*}$ using the heaviside step function

We first need to define some xo and zo

If we choose the values where xo=-2.5 and zo=-1 we need to figure out which index in the xb array and zb array refer to these values. 

Here is my first attempt at doing this, where I found the value closest to -2.5 and -1 by hand by looking through the arrays for xo and zo respectively. This is not the best procedure.

In [15]:
indexxo = findfirst(x -> x == -2.5390625, xb)
indexzo = findfirst(z -> z == -1.0546875, zb)

println("The value ∼ -2.5 is found at index $indexxo")
println("The value ∼ -1 is found at index $indexzo")

The value ∼ -2.5 is found at index 32
The value ∼ -1 is found at index 51


Let's try another method, taking advantage of the absolute value function instead. Subtract our chosen value of xo from each element of the xb array. Then take the absolute value of this array and use the findmin method to find the index where the deviation from -2.5 is the smallest. Repeat for z!

In [16]:
xo_val=-2.5
zo_val=-1.0
index_xo = findmin(abs.(xb.-xo_val))
index_zo = findmin(abs.(zb.-zo_val))

(0.0234375, 52)

Cool! We found that the value closest to -2.5 for xo is found at the 32nd index. This is the same result as above!

BUT, the computer caught my human error from the previous method. The value closest to -1 for zo is actually the 52nd index. 

### Now we know $x_o = 32$ and $z_o = 52$ and we can use this to calculate the Heaviside

$ H(\rho(x,t)-\rho(x_o, t)) = \left\{
\begin{array}{ll}
        0, \rho(x,t)\leq\rho(x_o, t)\\
        1, \rho(x,t)>\rho(x_o,t)\\
\end{array}
\right.$

#### Since our model outputs buoyancy values rather than density and we know $b \propto -\rho$ We can multiply both sides of the inequality by -1. 

$ H(b(x_o,t)-b(x, t)) = \left\{
\begin{array}{ll}
        0, b(x,t)\geq b(x_o, t)\\
        1, b(x,t)<b(x_o,t)\\
\end{array}
\right.$

In [ ]:
xo=32
zo=52

#Calculate the Heaviside both two methods below result in diff values

H = (Bₙ .>= Bₙ[xo,zo])

### Now, we use the following equation to find $z_{*}$.

#### $z_{*} = \frac{\int H dV}{A} $ The integral represents the volume in the fluid where the buoyancy is less than buoyancy at 
$x_o$ Therefore, $V(b<b_o) < V_{total}$ Additionally, 
$dV = dx dy dz$ and since dy is not dependent on H, it can be taken out of the integral. Also, $A=L_x L_y$ Thus, we can cancel $L_y$ and rewrite the integral as: $z_{*} = \frac{\int H dx dz}{L_x}$

To solve the integral, we can breakdown our function and essentially take the sum of area under the curve. $\int H dA = \sum_{i,j \epsilon [0,N_x;0,N_z]} H \Delta A$ 

Here, $\Delta A = \Delta x \Delta z$ where $\Delta x = \frac{L_x}{N_x}$ and $\Delta z = \frac{L_z}{N_z}$. 

$L_x = L_z = 10$ is the prescribed length of the grid and $N_x = N_z = 128$ is the number of grid cells within the entire grid. Therefore $\Delta A = \frac{10^2}{128^2}$

In [ ]:
deltaA = fill(10^2/128^2, 128)
Lx=10

z_star = sum(H .*deltaA)/Lx

#z_star is a scalar....

### Make a plot of matrix H in the x versus buoyancy in the x


In [ ]:
fstep = Figure()
Axstep = Axis(fstep[1,1], title="Plot of Heaviside Step Function versus Buoyancy for xo=-2.5 and zo=-1", xlabel="Buoyancy", ylabel="Heaviside")
lines!(Axstep, Bₙ[1,:], H[1,:])

fstep

Make a heatmap for xo=-2.5, zo=-1

In [ ]:
f1 = Figure()
ax1 = Axis(f1[1,1],title="Heatmap for xo=-2.5 and zo=-1", xlabel="x[m]", ylabel="z[m]")
heatmap!(ax1,xb,zb,H)
f1

### Now, we must repeat this process for every element in xb and zb to create the vector $\vec{z_{*}}$